In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
%load_ext tensorboard

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [4]:
from PIL import Image
from IPython.display import display

import os
import numpy as np
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

from get_frames import get_frames
from build_model_ed import build_model_1, build_model_2
from build_model_lstm import build_model_lstm_1, build_model_lstm_2, build_model_lstm_3
from create_encoder_decoder_data import process_image, process_dataset, get_full_path
from get_encoder_decoder import get_encoder_decoder

In [5]:
#gpu check
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [6]:
#load data
unit_numb = 5
test_size = 0.1
stride_step = 50
norm_factor=1
dtype = 'uint8'
file_name = f'save_data//encoded_model_2//encoded_{unit_numb}_{stride_step}_ts_{test_size}_{dtype}_norm_{norm_factor}.pkl'
with open(file_name, 'rb') as f:
    frames_data_encoded = pickle.load(f)
    
print(f'Data shape: {frames_data_encoded.shape}')

Data shape: (770, 80, 5)


In [7]:
#shuffle
np.random.shuffle(frames_data_encoded)

Initial data shape: (770, 80, 5)
Input data shape: (55440, 6, 5)
Output data shape: (55440, 6, 5)


In [9]:
cells_list = [20,20]
ifDense = True
input_shape = (None, unit_numb)
many_to_many = True
loss = 'mean_squared_error'
learning_rate = 0.1
optimizer_name = "Adadelta"
optimizer = tf.keras.optimizers.legacy.Adadelta(learning_rate=learning_rate, name=optimizer_name)
#optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

model = build_model_lstm_3(input_shape, cells_list, ifDense=ifDense, ifDropout=False, many_to_many=many_to_many)
#model.stateful = True
model.compile(loss=loss, 
              optimizer=optimizer, )
              #metrics=metrics)
    

model.summary() 

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 5)]         0         
                                                                 
 lstm (LSTM)                 (None, None, 20)          2080      
                                                                 
 lstm_1 (LSTM)               (None, None, 20)          3280      
                                                                 
 dense (Dense)               (None, None, 5)           105       
                                                                 
Total params: 5465 (21.35 KB)
Trainable params: 5465 (21.35 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
model.stateful

False

In [11]:
#check activation function of the lstm layer
model.layers[1].activation

<function keras.src.activations.tanh(x)>

In [12]:
#model training 16 ifDense False bp 

folder = 'lstm_window_many_to_many'
epochs = 2000
batch_size = 72
validation_split = 0.1

cells_list_str = '_'.join(str(x) for x in cells_list)

model_name = f'model_lstm_window_{window_size_predicted}_units_{unit_numb}_bs2_{input_shape[-1]}_{cells_list_str}_ifDense_{ifDense}'
model_name = f'model_lstm_window_{window_size_predicted}_overlap_{overlap}_units_{unit_numb}_bs2_{input_shape[-1]}_{cells_list_str}_ifDense_{ifDense}'


#define callbacks
# Write TensorBoard logs
log_dir = f'./tensorboard/{folder}/logs/{model_name}'
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
#Stop training when no improvement
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                  patience=6, 
                                                  restore_best_weights=True)
#Reduce learning rate when a metric has stopped improving
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=3)
callbacks = [tensorboard, early_stopping, reduce_lr]

In [13]:
# Clear any logs from previous runs
if 1:
    ld = log_dir
else:
    ld=None
!rm -rf $ld

In [19]:
#bs 10 [10,10]
model.fit(
    x_train_lstm,
    y_train_lstm,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
    callbacks=callbacks,
)

Epoch 1/2000
4990/4990 [==============================] - 16s 3ms/step - loss: 0.0366 - val_loss: 0.0339 - lr: 0.1000
Epoch 2/2000
4990/4990 [==============================] - 13s 3ms/step - loss: 0.0267 - val_loss: 0.0222 - lr: 0.1000
Epoch 3/2000
4990/4990 [==============================] - 15s 3ms/step - loss: 0.0183 - val_loss: 0.0167 - lr: 0.1000
Epoch 4/2000
4990/4990 [==============================] - 14s 3ms/step - loss: 0.0140 - val_loss: 0.0127 - lr: 0.1000
Epoch 5/2000
4990/4990 [==============================] - 14s 3ms/step - loss: 0.0115 - val_loss: 0.0112 - lr: 0.1000
Epoch 6/2000
4990/4990 [==============================] - 14s 3ms/step - loss: 0.0103 - val_loss: 0.0099 - lr: 0.1000
Epoch 7/2000
4990/4990 [==============================] - 13s 3ms/step - loss: 0.0089 - val_loss: 0.0083 - lr: 0.1000
Epoch 8/2000
4990/4990 [==============================] - 13s 3ms/step - loss: 0.0073 - val_loss: 0.0067 - lr: 0.1000
Epoch 9/2000
4990/4990 [==============================] 

In [24]:
model.save_weights(f'saved_models/{folder}/{model_name}/{model_name}')

In [11]:
#tensorboard results
folder = 'lstm_window'
cells_list = [10,10,10]
cells_list_str = '_'.join(str(x) for x in cells_list)


model_name = f'model_lstm_window_units_{unit_numb}_bs2_{input_shape[-1]}_{cells_list_str}_ifDense_{ifDense}'

log_dir = f'./tensorboard/{folder}/logs/{model_name}'
%tensorboard --logdir $log_dir